# P2WSH MultiSig BIP322
## Role: Creator / Updator (see role definitions [here](https://github.com/bitcoin/bips/blob/master/bip-0174.mediawiki#Roles))

## Workflow diagram

Taken from [BIP174: Partially Signed Bitcoin Transaction Format](https://github.com/bitcoin/bips/blob/master/bip-0174.mediawik)

<img src="https://github.com/bitcoin/bips/raw/master/bip-0174/multisig-workflow.svg">


In [1]:
cd ..

/home/will/work/LegendaryRequirements/clients/dcd/bip0322-sigs


## 1. Create HD Key from mnemonic

Note: this would typically access some wallet

In [2]:
from buidl.mnemonic import secure_mnemonic
from buidl.hd import HDPrivateKey


In [3]:
mnemonic = 'priority orange artist purpose sport blade spike leg calm renew dress figure session play solve fatal spy reform humble carry finger response order print'

In [4]:
hd_rootkey = HDPrivateKey.from_mnemonic(mnemonic)

root_fp = hd_rootkey.fingerprint().hex()

## 2. Select Key to Sign With

In [5]:
bip32_path = "m/45'/0/0/6"

signing_privkey= hd_rootkey.traverse(bip32_path)
signing_pubkey = signing_privkey.pub.point


## 3. Fetch Other Signing Parties Public Keys

See [signer_1](./Signer_1.ipynb) and [signer_2](./Signer_2.ipynb) notebooks steps 1 to 3

In [6]:
signer_1_pubkey = '03c2a23919504eb85905cd03861f97931314808f667f2c357f85b72f00f41d9493'
signer_2_pubkey = '035ec5bae25dfb8c06ba3f996d5c44b056a94ca00ec3285fc3daa9721e7624ac5b'


## 4. Create 3-of-3 MultiSig Pay-to-Witness-Script Hash

Note: This p2wsh_address would likely be something that the signers would want to include in a DID Document.

The address of a p2wsh scriptPubKey is OP_0 plus a sha256 hash of the witness script. The witness script is equivalent to the redeemscript used for a p2sh multisig

In [7]:
from buidl.script import RedeemScript, WitnessScript

# YOU COULD CHANGE THIS
quorum_m=3

redeem_script = RedeemScript.create_p2sh_multisig(
            quorum_m=quorum_m,
            pubkey_hexes=[signing_pubkey.sec().hex(),signer_1_pubkey,signer_2_pubkey],
            sort_keys=False,
        )



witness_script = WitnessScript.convert(redeem_script.raw_serialize())
p2wsh_script_pubkey = witness_script.script_pubkey()
p2wsh_address = witness_script.address()
# segwit_script_pubkey.address()
p2wsh_script_pubkey

OP_0 f817c1077d10546de38bfb98f031086b33b3b3b7b62318f21925510b71079f17 

In [8]:
redeem_script

OP_3 03f77725af4d98d410d589a3031606c42bad7094b36d06bb9605efd086e466b45d 03c2a23919504eb85905cd03861f97931314808f667f2c357f85b72f00f41d9493 035ec5bae25dfb8c06ba3f996d5c44b056a94ca00ec3285fc3daa9721e7624ac5b OP_3 OP_CHECKMULTISIG 

In [9]:
witness_script

OP_3 03f77725af4d98d410d589a3031606c42bad7094b36d06bb9605efd086e466b45d 03c2a23919504eb85905cd03861f97931314808f667f2c357f85b72f00f41d9493 035ec5bae25dfb8c06ba3f996d5c44b056a94ca00ec3285fc3daa9721e7624ac5b OP_3 OP_CHECKMULTISIG 

In [10]:
print("The BIP322 to_spend tx will be `locked` with a script_pubkey with the following address : ", p2wsh_address)

The BIP322 to_spend tx will be `locked` with a script_pubkey with the following address :  bc1qlqtuzpmazp2xmcutlwv0qvggdvem8vahkc333usey4gskug8nutsz53msw


## 5. Create the virtual to_spend transaction locked with the p2wsh_multisig

BIP 0322 defines the structure of this transaction as follows:

```python
    nVersion = 0
    nLockTime = 0
    vin[0].prevout.hash = 0000...000
    vin[0].prevout.n = 0xFFFFFFFF
    vin[0].nSequence = 0
    vin[0].scriptSig = OP_0 PUSH32[ message_hash ]
    vin[0].scriptWitness = []
    vout[0].nValue = 0
    vout[0].scriptPubKey = message_challenge
```

The message_challenge is the above 3-3 p2wsh scriptPubKey

In [11]:
from src.message import create_to_spend_tx

## You could change this
message_to_sign = "This will be a p2wsh 3-of-3 multisig BIP 322 signed message"

to_spend_tx = create_to_spend_tx(p2wsh_address, message_to_sign)

## 6. Create PSBT to_sign transaction

In [12]:
from src.message import create_to_sign_tx

to_sign_tx = create_to_sign_tx(to_spend_tx.hash())

# NOTE: This is a bug in the current create_to_sign_tx function
# It SHOULD be able to know if it is a segwit transaction or not
to_sign_tx.segwit = False

In [13]:
from buidl.psbt_helper import create_multisig_psbt
from buidl.psbt import PSBT

fee = 0
# psbt_to_sign_bip322 = create_multisig_psbt(pubkey_records,input_dicts,output_dicts,fee)
psbt_to_sign_bip322 = PSBT.create(to_sign_tx)

In [14]:
psbt_to_sign_bip322.validate()

True

## 7. Update PSBT with data know by Creator

This includes:
    
- A lookup for the to_spend_tx referenced as an input to the to_sign_tx
- A lookup for creators HD Public key that is part of the p2sh multisig
- A lookup for the redeemscript associated with the p2sh multsig stored in the scriptPubKey of the TxOut for the to_spend_tx

### 7.1 Define the tx lookup

This is a map of a transaction has to the actual transaction object

In [30]:
tx_lookup = {}

tx_lookup[to_spend_tx.hash()] = to_spend_tx

### 7.2 Define a pubkey_lookup

This maps from the sec representation of a S256Point to a NamedHDPublicKey object.

Note: a NamedHDPublicKey is a HDPublicKey with additioanl raw path data for the fingerprint of the root HDPublicKey and the derivation path from that root.

In [31]:
from buidl.psbt import NamedHDPublicKey

pubkey_lookup = {}

named_hd_pubkey_obj = NamedHDPublicKey.from_hd_pub(
    child_hd_pub=signing_privkey.pub,
    xfp_hex=root_fp,
    path=bip32_path,
)
# pubkey lookups needed for validation
pubkey_lookup[named_hd_pubkey_obj.sec()] = named_hd_pubkey_obj

### 7.3 Define witness_lookup

This is a map from a sha256 of a witness script to the actual witness script

In [32]:
witness_lookup = {}
witness_lookup[witness_script.sha256()] = witness_script

In [33]:
# This is not set until after the update
psbt_to_sign_bip322.tx_obj.tx_ins[0]._script_pubkey

OP_0 f817c1077d10546de38bfb98f031086b33b3b3b7b62318f21925510b71079f17 

In [34]:
psbt_to_sign_bip322.update(tx_lookup,pubkey_lookup,None,witness_lookup)

In [35]:
# As you can see update populates the scriptPubKey from the to_spend transaction into the tx_obj in the PSBT 
# (the to_sign obj)
psbt_to_sign_bip322.tx_obj.tx_ins[0]._script_pubkey

OP_0 f817c1077d10546de38bfb98f031086b33b3b3b7b62318f21925510b71079f17 

## 8. Serialize and Share PSBT with Other Signing Parties

See 

In [36]:
psbt_to_sign_bip322.psbt_ins[0].prev_tx

In [26]:
from io import BytesIO

b64_psbt = psbt_to_sign_bip322.serialize_base64()

print("Copy this vvv \n\n")

print(f"b64_psbt = '{b64_psbt}'")

Copy this vvv 


b64_psbt = 'cHNidP8BAD0AAAAAAeTHmyyL0BtEMKgIYJ4VxLouFJf/k9NLF6+gu5yMrmP7AAAAAAAAAAAAAQAAAAAAAAAAAWoAAAAAAAEBKwAAAAAAAAAAIgAg+BfBB30QVG3ji/uY8DEIazOzs7e2IxjyGSVRC3EHnxcBBWlTIQP3dyWvTZjUENWJowMWBsQrrXCUs20Gu5YF79CG5Ga0XSEDwqI5GVBOuFkFzQOGH5eTExSAj2Z/LDV/hbcvAPQdlJMhA17FuuJd+4wGuj+ZbVxEsFapTKAOwyhfw9qpch52JKxbU64iBgP3dyWvTZjUENWJowMWBsQrrXCUs20Gu5YF79CG5Ga0XRR7BeobLQAAgAAAAAAAAAAABgAAAAAA'


## 9. Sign PSBT

In [44]:
psbt_to_sign_bip322.sign(hd_rootkey)

True

## 10. Recieve Signed PSBTs from Signer 1 and Signer 2 Notebooks

Note: in this case the 2-3 multisig is satisfied from signatures provided by signer_1 and signer_2. The Creator has no contributed a signature.

You could easily change this notebook so that it is the creator's signature plus one of the signers that satisfy the p2sh multisig instead though.

In [45]:
# Copy this from signer_1.ipynb

signer_1_psbt_b64 = 'cHNidP8BAD0AAAAAAeTHmyyL0BtEMKgIYJ4VxLouFJf/k9NLF6+gu5yMrmP7AAAAAAAAAAAAAQAAAAAAAAAAAWoAAAAAAAEBKwAAAAAAAAAAIgAg+BfBB30QVG3ji/uY8DEIazOzs7e2IxjyGSVRC3EHnxciAgPCojkZUE64WQXNA4Yfl5MTFICPZn8sNX+Fty8A9B2Uk0cwRAIgE3oMsFZehlqxvYybj6TcLYByslWAl9sn7kkg0RQEwGkCIEy5VOq3i0LEiK0Ehdtc9z7VDtsgFLHlFhmakVCBO1SqAQEFaVMhA/d3Ja9NmNQQ1YmjAxYGxCutcJSzbQa7lgXv0IbkZrRdIQPCojkZUE64WQXNA4Yfl5MTFICPZn8sNX+Fty8A9B2UkyEDXsW64l37jAa6P5ltXESwVqlMoA7DKF/D2qlyHnYkrFtTriIGA8KiORlQTrhZBc0Dhh+XkxMUgI9mfyw1f4W3LwD0HZSTFAAAAAAtAACAAAAAAAAAAAAJAAAAIgYD93clr02Y1BDViaMDFgbEK61wlLNtBruWBe/QhuRmtF0UewXqGy0AAIAAAAAAAAAAAAYAAAAAAA=='

In [46]:
signer_1_psbt = PSBT.parse_base64(signer_1_psbt_b64)

In [47]:
# Copy this from signer_2.ipynb

signer_2_psbt_b64 = 'cHNidP8BAD0AAAAAAeTHmyyL0BtEMKgIYJ4VxLouFJf/k9NLF6+gu5yMrmP7AAAAAAAAAAAAAQAAAAAAAAAAAWoAAAAAAAEBKwAAAAAAAAAAIgAg+BfBB30QVG3ji/uY8DEIazOzs7e2IxjyGSVRC3EHnxciAgNexbriXfuMBro/mW1cRLBWqUygDsMoX8PaqXIediSsW0gwRQIhAIMbaYnbFNGeLv/uUQfXMnkLaAluTXK3iyZll2fJFwmTAiAUdbF6BIp4WROAhfT/UkxiHDLwrmY2OheRdF01S4/UowEBBWlTIQP3dyWvTZjUENWJowMWBsQrrXCUs20Gu5YF79CG5Ga0XSEDwqI5GVBOuFkFzQOGH5eTExSAj2Z/LDV/hbcvAPQdlJMhA17FuuJd+4wGuj+ZbVxEsFapTKAOwyhfw9qpch52JKxbU64iBgNexbriXfuMBro/mW1cRLBWqUygDsMoX8PaqXIediSsWxQAHQ2eLQAAgAAAAAAAAAAACwAAACIGA/d3Ja9NmNQQ1YmjAxYGxCutcJSzbQa7lgXv0IbkZrRdFHsF6hstAACAAAAAAAAAAAAGAAAAAAA='

In [48]:
signer_2_psbt = PSBT.parse_base64(signer_2_psbt_b64)

## 11. Combine PSBTs 

In [49]:
# Before combination the psbt only has a single sig
len(psbt_to_sign_bip322.psbt_ins[0].sigs)

1

In [50]:
psbt_to_sign_bip322.combine(signer_1_psbt)

In [51]:
psbt_to_sign_bip322.combine(signer_2_psbt)

In [53]:
# Afterwards there are three signatures
len(psbt_to_sign_bip322.psbt_ins[0].sigs)

3

## 12. Finalize the PSBT to produce the scriptSig on the PSBT Input from the sigs

In [56]:
# ScriptSig not populated until after finalization
psbt_to_sign_bip322.psbt_ins[0].witness

In [57]:
psbt_to_sign_bip322.finalize()

In [59]:
psbt_to_sign_bip322.psbt_ins[0].witness

<null> 3045022100d0a17bc628b1f9f1ec6eba304eb5afbbf8637bbe2d252b0890c73069b8db97b80220204d176109450ce7f16de806618ce57feb6a7a095e00a13a4976086b553eb82701 30440220137a0cb0565e865ab1bd8c9b8fa4dc2d8072b2558097db27ee4920d11404c06902204cb954eab78b42c488ad0485db5cf73ed50edb2014b1e516199a9150813b54aa01 3045022100831b6989db14d19e2effee5107d732790b68096e4d72b78b26659767c917099302201475b17a048a7859138085f4ff524c621c32f0ae66363a1791745d354b8fd4a301 532103f77725af4d98d410d589a3031606c42bad7094b36d06bb9605efd086e466b45d2103c2a23919504eb85905cd03861f97931314808f667f2c357f85b72f00f41d949321035ec5bae25dfb8c06ba3f996d5c44b056a94ca00ec3285fc3daa9721e7624ac5b53ae 

## 13. Create Final Bitcoin Tx Object from PSBT

In [60]:
# Remember: currently the final_tx() function on the PSBT fails because there are no funds for a tx fee
# BUT for a bip322 we don't need any fee
psbt_to_sign_bip322.final_tx()

This transaction won't relay without having a fee of at least 143


RuntimeError: transaction invalid

In [61]:
## Proposed update to the final_tx function in psbt.py

def final_tx(psbt, is_bip322=False):
        """Returns the broadcast-able transaction"""
        # clone the transaction from self.tx_obj
        tx_obj = psbt.tx_obj.clone()
        # determine if the transaction is segwit by looking for a witness field
        #  in any PSBTIn. if so, set tx_obj.segwit = True
        if any([psbt_in.witness for psbt_in in psbt.psbt_ins]):
            tx_obj.segwit = True
        # iterate through the transaction and PSBT inputs together
        #  using zip(tx_obj.tx_ins, self.psbt_ins)
        for tx_in, psbt_in in zip(tx_obj.tx_ins, psbt.psbt_ins):
            # set the ScriptSig of the transaction input
            tx_in.script_sig = psbt_in.script_sig
            # Exercise 7: if the tx is segwit, set the witness as well
            if tx_obj.segwit:
                # witness should be the PSBTIn witness or an empty Witness()
                tx_in.witness = psbt_in.witness or Witness()
        # check to see that the transaction verifies
        if not is_bip322 and not tx_obj.verify():
            raise RuntimeError("transaction invalid")
        # return the now filled in transaction
        return tx_obj

In [62]:
final_to_sign_tx = final_tx(psbt_to_sign_bip322, True)

## 14. Check Signature on Input is Valid

In [63]:
final_to_sign_tx.verify_input(0)

True

## 15. Serialize to_sign transaction as BIP322 Signature

In [64]:
from buidl.helper import base64_encode
bip322_musig = base64_encode(final_to_sign_tx.serialize_witness())

print(bip322_musig)

BQBIMEUCIQDQoXvGKLH58exuujBOta+7+GN7vi0lKwiQxzBpuNuXuAIgIE0XYQlFDOfxbegGYYzlf+tqegleAKE6SXYIa1U+uCcBRzBEAiATegywVl6GWrG9jJuPpNwtgHKyVYCX2yfuSSDRFATAaQIgTLlU6reLQsSIrQSF21z3PtUO2yAUseUWGZqRUIE7VKoBSDBFAiEAgxtpidsU0Z4u/+5RB9cyeQtoCW5NcreLJmWXZ8kXCZMCIBR1sXoEinhZE4CF9P9STGIcMvCuZjY6F5F0XTVLj9SjAWlTIQP3dyWvTZjUENWJowMWBsQrrXCUs20Gu5YF79CG5Ga0XSEDwqI5GVBOuFkFzQOGH5eTExSAj2Z/LDV/hbcvAPQdlJMhA17FuuJd+4wGuj+ZbVxEsFapTKAOwyhfw9qpch52JKxbU64=


## 16. Verify BIP322 Signature against message and address

In [65]:
from src.message import verify_message

In [66]:
verify_message(p2wsh_address, bip322_musig, message_to_sign)

mismatch between length and consumed bytes 33 vs 32
mismatch between length and consumed bytes 93 vs 76
mismatch between length and consumed bytes 51 vs 16


True